In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm1(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm1(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [6]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += criterion(decoder_output[:, :2], target[:, di, :2])

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()*_input.size(0) / (counter * train_loader.batch_size)))

In [7]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 1

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [87]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(4):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [100]:
output = infer_absolute(test_loader, encoder, decoder)

In [102]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
10002,"[1715.350830078125, 338.973876953125]","[1716.6259765625, 339.9035339355469]","[1717.2083740234375, 339.84429931640625]","[1717.74658203125, 339.6815490722656]","[1718.30419921875, 339.5188293457031]","[1718.908203125, 339.4388427734375]","[1719.6080322265625, 339.6131591796875]","[1720.7086181640625, 340.97509765625]","[1722.5374755859375, 344.541748046875]","[1723.8411865234375, 346.6946105957031]",...,"[1734.8958740234375, 355.97198486328125]","[1736.0499267578125, 356.90478515625]","[1737.1007080078125, 357.67596435546875]","[1738.032958984375, 358.28875732421875]","[1738.8463134765625, 358.7608947753906]","[1739.5501708984375, 359.11676025390625]","[1740.1588134765625, 359.3810729980469]","[1740.68701171875, 359.5755920410156]","[1741.148681640625, 359.7179870605469]","[1741.56201171875, 359.8408508300781]"
10015,"[726.8098754882812, 1230.649658203125]","[724.795654296875, 1229.5281982421875]","[726.24853515625, 1229.7889404296875]","[724.8365478515625, 1228.835205078125]","[725.9271240234375, 1228.9228515625]","[724.8728637695312, 1228.1923828125]","[725.7037353515625, 1228.250732421875]","[724.9149780273438, 1227.7264404296875]","[725.5151977539062, 1227.740478515625]","[724.9458618164062, 1227.3612060546875]",...,"[724.3847045898438, 1224.2789306640625]","[724.28955078125, 1223.86376953125]","[724.1972045898438, 1223.4371337890625]","[724.1076049804688, 1223.005126953125]","[724.0224609375, 1222.5728759765625]","[723.94287109375, 1222.1455078125]","[723.8697509765625, 1221.7275390625]","[723.8037719726562, 1221.322509765625]","[723.7454223632812, 1220.9334716796875]","[723.6948852539062, 1220.56201171875]"
10019,"[575.8062133789062, 1244.832763671875]","[575.3055419921875, 1243.726318359375]","[575.4666748046875, 1243.8292236328125]","[575.7510986328125, 1243.8372802734375]","[576.0275268554688, 1243.790283203125]","[576.3048095703125, 1243.73095703125]","[576.5867919921875, 1243.6649169921875]","[576.8739624023438, 1243.5943603515625]","[577.1670532226562, 1243.521484375]","[577.4672241210938, 1243.4486083984375]",...,"[581.51171875, 1243.257080078125]","[581.948974609375, 1243.34326171875]","[582.392822265625, 1243.4508056640625]","[582.8410034179688, 1243.579345703125]","[583.2913208007812, 1243.7276611328125]","[583.74169921875, 1243.8941650390625]","[584.1902465820312, 1244.0767822265625]","[584.6353149414062, 1244.2730712890625]","[585.0753173828125, 1244.4803466796875]","[585.509033203125, 1244.69580078125]"
10028,"[1691.386962890625, 316.22113037109375]","[1692.322265625, 316.3926086425781]","[1692.64892578125, 316.0973205566406]","[1693.0811767578125, 315.9586486816406]","[1693.688720703125, 316.4206848144531]","[1694.9542236328125, 319.0085754394531]","[1695.9344482421875, 321.2515869140625]","[1696.3646240234375, 321.8315124511719]","[1696.7259521484375, 322.0668029785156]","[1697.1044921875, 322.2877197265625]",...,"[1704.691162109375, 329.6659851074219]","[1705.4278564453125, 330.331298828125]","[1706.1104736328125, 330.89617919921875]","[1706.736572265625, 331.3598327636719]","[1707.3074951171875, 331.7286376953125]","[1707.8272705078125, 332.0134582519531]","[1708.301513671875, 332.2271423339844]","[1708.735595703125, 332.3826904296875]","[1709.13525390625, 332.4918518066406]","[1709.5052490234375, 332.56475830078125]"
1003,"[2120.589111328125, 675.1353149414062]","[2119.99462890625, 674.2186889648438]","[2119.0341796875, 673.8532104492188]","[2117.910888671875, 672.6499633789062]","[2116.644287109375, 671.6104125976562]","[2115.373291015625, 670.5000610351562]","[2113.887451171875, 669.3497314453125]","[2112.24853515625, 668.166015625]","[2110.923828125, 667.2268676757812]","[2109.90869140625, 666.5438842773438]",...,"[2097.24072265625, 652.824462890625]","[2093.54150390625, 650.168701171875]","[2091.30712890625, 649.2930908203125]","[2090.764404296875, 648.927734375]","[2090.45361328125, 648.95458984375]","[2090.24267578125, 648.767272949218

In [104]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [105]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
10002,1715.350830,338.973877,1716.625977,339.903534,1717.208374,339.844299,1717.746582,339.681549,1718.304199,339.518829,...,1739.550171,359.116760,1740.158813,359.381073,1740.687012,359.575592,1741.148682,359.717987,1741.562012,359.840851
10015,726.809875,1230.649658,724.795654,1229.528198,726.248535,1229.788940,724.836548,1228.835205,725.927124,1228.922852,...,723.942871,1222.145508,723.869751,1221.727539,723.803772,1221.322510,723.745422,1220.933472,723.694885,1220.562012
10019,575.806213,1244.832764,575.305542,1243.726318,575.466675,1243.829224,575.751099,1243.837280,576.027527,1243.790283,...,583.741699,1243.894165,584.190247,1244.076782,584.635315,1244.273071,585.075317,1244.480347,585.509033,1244.695801
10028,1691.386963,316.221130,1692.322266,316.392609,1692.648926,316.097321,1693.081177,315.958649,1693.688721,316.420685,...,1707.827271,332.013458,1708.301514,332.227142,1708.735596,332.382690,1709.135254,332.491852,1709.505249,332.564758
1003,2120.589111,675.135315,2119.994629,674.218689,2119.034180,673.853210,2117.910889,672.649963,2116.644287,671.610413,...,2090.242676,648.767273,2089.811035,648.036011,2088.587891,645.095032,2086.752197,642.616028,2085.874023,642.217773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9897,258.256470,805.223083,257.811005,804.683533,257.838806,804.702515,258.136627,804.653809,258.442749,804.497681,...,264.782532,800.452881,265.098694,800.340088,265.413422,800.240601,265.726349,800.154785,266.037201,800.083069
99,588.930054,1155.061768,587.916199,1154.400269,588.755981,1153.956299,588.094666,1153.111816,588.661255,1152.707520,...,588.467163,1141.150024,588.504211,1140.643799,588.536255,1140.112183,588.562317,1139.550537,588.581909,1138.955566
9905,1755.729126,443.640533,1755.197876,444.483673,1755.113281,445.519867,1754.667725,446.108093,1754.495117,446.723267,...,1749.704590,454.435638,1749.489868,454.679718,1749.273926,454.924011,1749.056763,455.169464,1748.838013,455.417053


In [106]:
df2.to_csv("outputs.csv", index=True)